### `Homework4: deployment`


## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [1]:
# To install Pipenv and check its version
pip install pipenv

In [ ]:
pipenv --version

pipenv, version 2024.2.0

## Question 2

* Use Pipenv to install Scikit-Learn version 1.5.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

> **Note**: you should create an empty folder for homework and do it there. 

In [ ]:
pipenv install scikit-learn==1.5.2

```
"scikit-learn": {
            "hashes": [
                "sha256:03b6158efa3faaf1feea3faa884c840ebd61b6484167c711548fce208ea09445",

```

## Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

```python
features = ['job', 'duration', 'poutcome']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression().fit(X, y)
```

> **Note**: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

* [DictVectorizer](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2024/05-deployment/homework/dv.bin?raw=true)
* [LogisticRegression](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/cohorts/2024/05-deployment/homework/model1.bin?raw=true)

With `wget`:

```bash
PREFIX=https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/master/cohorts/2024/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin
```


## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"job": "management", "duration": 400, "poutcome": "success"}
```

What's the probability that this client will get a subscription? 

* 0.359
* 0.559
* 0.759
* 0.959

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum model1.bin dv.bin
3d8bb28974e55edefa000fe38fd3ed12  model1.bin
7d37616e00aa80f2152b8b0511fc2dff  dv.bin
```

In [8]:
import pickle

# Load the vectorizer and model
with open('dv.bin', 'rb') as f_dv, open('model1.bin', 'rb') as f_model:
    dv = pickle.load(f_dv)
    model = pickle.load(f_model)

# Client data
client = {"job": "management", "duration": 400, "poutcome": "success"}

# Transform the client data and make a prediction
X = dv.transform([client])
y_pred = model.predict_proba(X)[0, 1]

print("Probability of subscription:", y_pred)

c:\Users\Daiyaan\miniconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Probability of subscription: 0.7590966516879658


c:\Users\Daiyaan\miniconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


## Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"job": "student", "duration": 280, "poutcome": "failure"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription?

* 0.335
* 0.535
* 0.735
* 0.935




 ### Install Flask and Gunicorn            
 Run the following command in your terminal:
 > pipenv install flask gunicorn

 If you’re on Windows, you can install waitress instead of gunicorn:
 > pipenv install waitress

### Flask Code for Serving the Model
Create a new file named `app.py`

In [ ]:
import pickle
from flask import Flask, request, jsonify

app = Flask(__name__)

# Load the vectorizer and model
with open('dv.bin', 'rb') as f_dv, open('model1.bin', 'rb') as f_model:
    dv = pickle.load(f_dv)
    model = pickle.load(f_model)

@app.route('/predict', methods=['POST'])
def predict():
    client = request.json
    X = dv.transform([client])
    y_pred = model.predict_proba(X)[0, 1]
    return jsonify({'probability': y_pred})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

### Run the Flask Application
waitress-serve --host=0.0.0.0 --port=5000 app:app

### Test the client 
Use the following code to score the client using the requests library. You can run this in another Python script

In [41]:
import requests

url = "http://localhost:5000/predict"  # Adjust this if deploying elsewhere
client = {"job": "student", "duration": 280, "poutcome": "failure"}
response = requests.post(url, json=client)
probability = response.json()['probability']

print("Probability of subscription:", probability)


#### Run the `question4.py` script
Probability of subscription: 0.33480703475511053

## Docker

Install [Docker](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/05-deployment/06-docker.md). 
We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.11.5-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.11.5-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.11.5-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.11.5-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.

## Question 5

Download the base image `svizor/zoomcamp-model:3.11.5-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 45 MB
* 130 MB
* 245 MB
* 330 MB

You can get this information when running `docker images` - it'll be in the "SIZE" column.

- To find the size of the Docker image `svizor/zoomcamp-model:3.11.5-slim`, follow these steps:

    - Pull the Base Image (if you haven't done this already):

    - Open your terminal and run:

Download the base image `svizor/zoomcamp-model:3.11.5-slim` with docker pull command.

> docker image pull svizor/zoomcamp-model:3.11.5-slim

Run `docker images` command:

> REPOSITORY              TAG           IMAGE ID       CREATED      SIZE                      
> svizor/zoomcamp-model   3.11.5-slim   975e7bdca086   1 days ago   130MB

## Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```docker
FROM svizor/zoomcamp-model:3.11.5-slim

```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn 

After that, you can build your docker image.

## Question 6

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"job": "management", "duration": 400, "poutcome": "success"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a subscription now?

* 0.287
* 0.530
* 0.757
* 0.960